In [1]:
import pandas as pd

In [4]:
import os
import json
wd = os.getcwd()
wd_list = wd.split("/")
unique_path = "/".join(wd_list[1:3])
#I have the passwords file saved as 'nothing_to_see_here.json'. Super secure, I know
path = "/"+unique_path+"/esh/nothing_to_see_here.json"
#Load json file in .esh and get into creds
#This accesses for the db_creds dictionary specifically. Other dicts are available by changing the ref below
creds = json.load(open(path))['db_creds']
from sqlalchemy import create_engine
import os
###DONT SHARE WITH LOGIN DETAILS###
POSTGRES_ADDRESS = creds['esh_db_address']
POSTGRES_PORT =  creds['esh_db_port']
POSTGRES_USERNAME = creds['esh_db_username']
POSTGRES_PASSWORD = creds['esh_db_password']
POSTGRES_DBNAME = creds['esh_db_name']
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,
                password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
# Create the connection
cnx = create_engine(postgres_str)

In [6]:
query_hca = '''
SELECT 
state_id, 
SUM(CASE when high_cost_indicator = 'true'
	THEN total_households
   ELSE 0 END) as high_cost_households,
SUM(CASE WHEN high_cost_indicator = 'false'
   THEN total_households
   ELSE 0 END) as low_cost_households,
SUM(CASE when high_cost_indicator = 'true'
	THEN total_households
   ELSE 0 END) / SUM(total_households) as high_cost_pct,
SUM(total_households) as total_households
FROM dl.high_cost_census_block_groups
GROUP BY state_id
ORDER BY state_id

'''

In [3]:
query_elig = '''
WITH df as (
	SELECT 
		--identifying info
		hh.state_id, hh.puma20_id, hh.serialno,
		-- eligibility
		MAX(
			CASE WHEN (pop.hins4::text = '1'::text OR 
									hh.fs::text = '1'::text OR
									pop.pap > 0::numeric OR
									pop.ssip > 0::numeric OR 
									pop.povpip <= 200::numeric) 
			THEN 1
			ELSE 0 END) as pbb_eligible,
		--unconnected
		MAX(
			CASE WHEN  hh.accessinet::text = '3'::text OR
						(hh.accessinet::text = '1'::text AND 
						 hh.hispeed::text = '2'::text AND 
						 hh.othsvcex::text = '2'::text AND 
						 (hh.dialup::text = '1'::text OR hh.dialup::text = '2'::text) AND 
						 hh.satellite::text = '2'::text) 
			THEN 1
			ELSE 0 END) AS unconnected,
		MAX(hh.wgtp) as hh_weight
		FROM dl.pums_households_2022 hh
		LEFT JOIN dl.pums_population_2022 pop ON hh.puma20_id::text = pop.puma20_id::text AND hh.serialno::text = pop.serialno::text
	GROUP BY hh.state_id, hh.serialno, hh.puma20_id, 
		hh.accessinet, hh.hispeed, hh.othsvcex, hh.dialup, hh.satellite
)		
SELECT state_id,
SUM(CASE WHEN pbb_eligible = 1
	THEN hh_weight
	ELSE 0 END) AS hh_eligible, 
SUM(CASE WHEN unconnected = 1
	THEN hh_weight
	ELSE 0 END) AS hh_unconnected,
SUM(CASE WHEN pbb_eligible = 1 AND unconnected = 1
	THEN hh_weight
	ELSE 0 END) AS hh_eligible_unconnected

FROM df
GROUP BY state_id

'''

In [7]:
hca = pd.read_sql(query_hca, cnx)
hca.head()

,state_id,high_cost_households,low_cost_households,high_cost_pct,total_households
0,01,1904.0,1901079.0,0.001001,1902983.0
1,02,21280.0,239281.0,0.081670,260561.0
2,04,10525.0,2673032.0,0.003922,2683557.0
3,05,3725.0,1154735.0,0.003215,1158460.0
4,06,30108.0,13187478.0,0.002278,13217586.0


In [8]:
elig = pd.read_sql(query_elig, cnx)
elig.head()

,state_id,hh_eligible,hh_unconnected,hh_eligible_unconnected
0,01,907340.0,566705.0,355108.0
1,02,101493.0,67260.0,36491.0
2,04,1175293.0,538640.0,332124.0
3,05,605148.0,367410.0,238051.0
4,06,5989677.0,2300227.0,1461144.0


In [9]:
df = hca.merge(elig, on = 'state_id')
df.head()

,state_id,high_cost_households,low_cost_households,high_cost_pct,total_households,hh_eligible,hh_unconnected,hh_eligible_unconnected
0,01,1904.0,1901079.0,0.001001,1902983.0,907340.0,566705.0,355108.0
1,02,21280.0,239281.0,0.081670,260561.0,101493.0,67260.0,36491.0
2,04,10525.0,2673032.0,0.003922,2683557.0,1175293.0,538640.0,332124.0
3,05,3725.0,1154735.0,0.003215,1158460.0,605148.0,367410.0,238051.0
4,06,30108.0,13187478.0,0.002278,13217586.0,5989677.0,2300227.0,1461144.0


In [10]:
df['high_cost_eligible_unconnected'] = df['high_cost_pct']*df['hh_eligible_unconnected']
df['high_cost_eligible_unconnected'].sum()

23395.39693370843

In [11]:
df.head()

,state_id,high_cost_households,low_cost_households,high_cost_pct,total_households,hh_eligible,hh_unconnected,hh_eligible_unconnected,high_cost_eligible_unconnected
0,01,1904.0,1901079.0,0.001001,1902983.0,907340.0,566705.0,355108.0,355.297778
1,02,21280.0,239281.0,0.081670,260561.0,101493.0,67260.0,36491.0,2980.217607
2,04,10525.0,2673032.0,0.003922,2683557.0,1175293.0,538640.0,332124.0,1302.601398
3,05,3725.0,1154735.0,0.003215,1158460.0,605148.0,367410.0,238051.0,765.447210
4,06,30108.0,13187478.0,0.002278,13217586.0,5989677.0,2300227.0,1461144.0,3328.302426


In [12]:
df.to_csv('/Users/mhamburger94/ESH/Projects/PBB/pbb_budget_estimate.csv', index=False)